In [2]:
from importlib import reload

import language
import normalizer
import parser

reload(language)
reload(normalizer)
reload(parser)

from language import *
from normalizer import *
from parser import *

In [3]:
def print_normalized(term_str: str):
    term = parse_term(term_str)
    print(f"Parsed term: {term}")
    desugared = desugar(term)
    print(f"Desugared: {desugared}")
    db_term, fv = normal_to_deBruijn(desugared)
    print(f"de Bruijn indices: {db_term}")
    # print(f"free variables: {free_vars}")

    db_normalized = normalize_subst_no(db_term)
    print(f"deBruijn normalized: {db_normalized}")

    normalized = deBruijn_to_normal(db_normalized, fv)
    print(f"normalized with variables: {normalized}")

In [4]:
print_normalized("lambda x. (lambda x.x) x y")
print("------------------------------------")
print_normalized("add 1 2")
print("------------------------------------")
print_normalized("head (cons 3 (cons 4 (cons 5 nil)))")
print("------------------------------------")
print_normalized("true false")

Parsed term: λx. ((λx. x) x) y
Desugared: λx. ((λx. x) x) y
de Bruijn indices: λ. ((λ. 0) 0) 1
deBruijn normalized: λ. 0 1
normalized with variables: λx0. x0 y
------------------------------------
Parsed term: add (1) (2)
Desugared: ((λm. λn. λs. λz. (m s) ((n s) z)) (λs. λz. s z)) (λs. λz. s (s z))
de Bruijn indices: ((λ. λ. λ. λ. (3 1) ((2 1) 0)) (λ. λ. 1 0)) (λ. λ. 1 (1 0))
deBruijn normalized: λ. λ. 1 (1 (1 0))
normalized with variables: λx0. λx1. x0 (x0 (x0 x1))
------------------------------------
Parsed term: head (cons (3) (cons (4) (cons (5) (nil))))
Desugared: (λl. (l (λh. λt. h)) (λt. λf. f)) (((λh. λt. λc. λn. (c h) ((t c) n)) (λs. λz. s (s (s z)))) (((λh. λt. λc. λn. (c h) ((t c) n)) (λs. λz. s (s (s (s z))))) (((λh. λt. λc. λn. (c h) ((t c) n)) (λs. λz. s (s (s (s (s z)))))) (λc. λn. n))))
de Bruijn indices: (λ. (0 (λ. λ. 1)) (λ. λ. 0)) (((λ. λ. λ. λ. (1 3) ((2 1) 0)) (λ. λ. 1 (1 (1 0)))) (((λ. λ. λ. λ. (1 3) ((2 1) 0)) (λ. λ. 1 (1 (1 (1 0))))) (((λ. λ. λ. λ. (1 3) ((2 1)

In [5]:
print_normalized("x y z true false")

Parsed term: (((x y) z) (true)) (false)
Desugared: (((x y) z) (λt. λf. t)) (λt. λf. f)
de Bruijn indices: (((0 1) 2) (λ. λ. 1)) (λ. λ. 0)
deBruijn normalized: (((0 1) 2) (λ. λ. 1)) (λ. λ. 0)
normalized with variables: (((x y) z) (λx0. λx1. x0)) (λx0. λx1. x1)


In [6]:
# print_normalized("true false")

In [7]:
print_normalized("lambda y. (lambda x. lambda y. y) y")

Parsed term: λy. (λx. λy. y) y
Desugared: λy. (λx. λy. y) y
de Bruijn indices: λ. (λ. λ. 0) 0
deBruijn normalized: λ. λ. 0
normalized with variables: λx0. λx1. x1


In [8]:
print_normalized("lambda y. (lambda x. lambda y. x) y")

Parsed term: λy. (λx. λy. x) y
Desugared: λy. (λx. λy. x) y
de Bruijn indices: λ. (λ. λ. 1) 0
deBruijn normalized: λ. λ. 1
normalized with variables: λx0. λx1. x0


In [9]:
def print_comparison(term_str_1, term_str_2):
    clo1 = parse_term_to_deBruijn(term_str_1)
    clo2 = parse_term_to_deBruijn(term_str_2)
    print(beta_equal(clo1, clo2))

In [11]:
print_normalized("lambda y.  (lambda x. lambda y. x) y")

Parsed term: λy. (λx. λy. x) y
Desugared: λy. (λx. λy. x) y
de Bruijn indices: λ. (λ. λ. 1) 0
deBruijn normalized: λ. λ. 1
normalized with variables: λx0. λx1. x0


In [12]:
print_normalized("add 1 0")
print_normalized("(lambda m. lambda n. lambda s. lambda z. (3 1) ((2 1) 0)) (lambda s. lambda z. 1 0)")
print_normalized("(lambda m. lambda n. lambda s. lambda z. (m s) ((n s) z)) (lambda s. lambda z. s z)")
print_normalized("add 5 0")

Parsed term: add (5) (0)
Desugared: ((λm. λn. λs. λz. (m s) ((n s) z)) (λs. λz. s (s (s (s (s z)))))) (λs. λz. z)
de Bruijn indices: ((λ. λ. λ. λ. (3 1) ((2 1) 0)) (λ. λ. 1 (1 (1 (1 (1 0)))))) (λ. λ. 0)
deBruijn normalized: λ. λ. 1 (1 (1 (1 (1 0))))
normalized with variables: λx0. λx1. x0 (x0 (x0 (x0 (x0 x1))))


In [13]:
print_comparison("lambda x. x", "lambda y. y")
print_comparison("lambda x. x", "lambda y. z")
print_comparison("head (cons 5 nil)", "add 2 3")

True
False
True
